In [100]:
import numpy as np
import scipy.io as spio
import random as rand
import math
from scipy.spatial.distance import squareform, cdist
import matplotlib.pylab as plt

In [128]:
mat = spio.loadmat('digits (3).mat')
trainImages = mat['trainImages'] 
trainLabels = mat['trainLabels'] 
testImages = mat['testImages'] 
testLabels = mat['testLabels'] 
length = 60000
test_length = 5000

X = trainImages.reshape(784,60000)
X = X[:,0:length].T
Y = trainLabels[:,0:length].T

X_test = testImages.reshape(784,10000)
X_test = X_test[:,0:test_length].T
Y_test = testLabels[:,0:test_length].T

print(len(X), X.shape, X.dtype, len(Y), Y.shape, Y.dtype)
print(len(X_test), X_test.shape, X_test.dtype, len(Y_test), Y_test.shape, Y_test.dtype)

60000 (60000, 784) uint8 60000 (60000, 1) uint8
5000 (5000, 784) uint8 5000 (5000, 1) uint8


In [129]:
Y = np.asarray(Y,dtype=np.float64)
Y_test = np.asarray(Y_test,dtype=np.float64)
# #0 vs rest(1----9)
# for i in range(0,len(Y)):
#     if Y[i][0]==0:
#         Y[i][0] = 1.0
#     else:
#         Y[i][0] = -1.0
# for i in range(0,len(Y_test)):
#     if Y_test[i][0]==0:
#         Y_test[i][0] = 1
#     else:
#         Y_test[i][0] = -1
        
# # 7 vs rest
# for i in range(0,len(Y)):
#     if Y[i][0]==7:
#         Y[i][0] = 1.0
#     else:
#         Y[i][0] = -1.0
# for i in range(0,len(Y_test)):
#     if Y_test[i][0]==7:
#         Y_test[i][0] = 1
#     else:
#         Y_test[i][0] = -1
# # 4 vs 9
# Y = Y.flatten()
# indices = np.where((Y==4) | (Y==9))
# indices4 = np.where(Y==4)
# indices9 = np.where(Y==9)
# np.put(Y, indices4, [1.0])
# np.put(Y, indices9, [-1.0])
# Y = np.take(Y,indices)
# X = X[indices, :]
# a,b,c = X.shape
# X = X.reshape(b,784)
# Y = Y.T

# Y_test = Y_test.flatten()
# t_indices = np.where((Y_test==4) | (Y_test==9))
# t_indices4 = np.where(Y_test==4)
# t_indices9 = np.where(Y_test==9)
# np.put(Y_test, t_indices4, [1.0])
# np.put(Y_test, t_indices9, [-1.0])
# Y_test = np.take(Y_test,t_indices)
# X_test = X_test[t_indices, :]
# d,e,f = X_test.shape
# print(X_test.shape)
# X_test = X_test.reshape(e,784)
# Y_test = Y_test.T

# #0 vs 8
Y = Y.flatten()
indices = np.where((Y==0) | (Y==8))
indices0 = np.where(Y==0)
indices8 = np.where(Y==8)
np.put(Y, indices0, [1.0])
np.put(Y, indices8, [-1.0])
Y = np.take(Y,indices)
X = X[indices, :]
a,b,c = X.shape
X = X.reshape(b,784)
Y = Y.T

Y_test = Y_test.flatten()
t_indices = np.where((Y_test==0) | (Y_test==8))
t_indices0 = np.where(Y_test==0)
t_indices8 = np.where(Y_test==8)
np.put(Y_test, t_indices0, [1.0])
np.put(Y_test, t_indices8, [-1.0])
Y_test = np.take(Y_test,t_indices)
X_test = X_test[t_indices, :]
d,e,f = X_test.shape
X_test = X_test.reshape(e,784)
Y_test = Y_test.T

# # #(0,8,3) vs (1,7,9)
# Y = Y.flatten()
# indices = np.where((Y==0) | (Y==8) | (Y==3) | (Y==1) | (Y==7) | (Y==9))
# indices083 = np.where((Y==0) | (Y==8) | (Y==3))
# indices179 = np.where((Y==1) | (Y==7) | (Y==9))
# np.put(Y, indices083, [1.0])
# np.put(Y, indices179, [-1.0])
# Y = np.take(Y,indices)
# X = X[indices, :]
# a,b,c = X.shape
# X = X.reshape(b,784)
# Y = Y.T

# Y_test = Y_test.flatten()
# t_indices = np.where((Y_test==0) | (Y_test==8) | (Y_test==3) | (Y_test==1) | (Y_test==7) | (Y_test==9))
# t_indices083 = np.where((Y_test==0) | (Y_test==8) | (Y_test==3))
# t_indices179 = np.where((Y_test==1) | (Y_test==7) | (Y_test==9))
# np.put(Y_test, t_indices083, [1.0])
# np.put(Y_test, t_indices179, [-1.0])
# Y_test = np.take(Y_test,t_indices)
# X_test = X_test[t_indices, :]
# d,e,f = X_test.shape
# X_test = X_test.reshape(e,784)
# Y_test = Y_test.T

print(len(X), X.shape, X.dtype, len(Y), Y.shape, Y.dtype)
print(len(X_test), X_test.shape, X_test.dtype, len(Y_test), Y_test.shape, Y_test.dtype)

11774 (11774, 784) uint8 11774 (11774, 1) float64
949 (949, 784) uint8 949 (949, 1) float64


In [130]:
def rbf_kernel(X_i, X_j, sigma = 0.5):
    pairwise_dists = cdist(X_i, X_j, 'euclidean')
    K = np.exp(-(pairwise_dists ** 2) / (2 * sigma * sigma))
    return np.matrix(K)

In [131]:
def kernel_polynomial(x1, x2):
    degree = 3
    return np.mat((np.dot(x1, x2.T) ** degree)) 

In [132]:
def fit(K='polynomial', C = 5.0, tol = 1e-5, max_passes=5):
    m, n = X.shape    
    alphas = np.matrix(np.zeros([m, 1]))
    b = 0.0
    
#     K_cache = kernel_polynomial(X, X)
    K_cache = rbf_kernel(X,X)
    iter = 0
    passes = 0
    while passes < max_passes:
        iter += 1
        num_changed_alphas = 0
        for i in range(m):
            temp = np.multiply(Y, K_cache[:, i])
            fx_i = alphas.T * np.multiply(Y, K_cache[:,i]) + b
            y_i = Y[i]
            E_i = fx_i - y_i
            alpha_i = alpha_ii = alphas[i]
            if (y_i * E_i < -tol and alpha_i < C) or (y_i * E_i > tol and alpha_i > 0.0):
                while True:
                    j = rand.randint(0, m - 1)
                    if i != j: break
                fx_j = alphas.T * np.multiply(Y, K_cache[:, j]) + b                
                y_j = Y[j]
                E_j = fx_j - y_j
                alpha_j = alpha_jj = alphas[j]
                if y_i != y_j:
                    L = max(0.0, alpha_j - alpha_i)
                    H = min(C, C + alpha_j - alpha_i)
                else:
                    L = max(0.0, alpha_i + alpha_j - C)
                    H = min(C, alpha_i + alpha_j)
                if L == H: continue

                eta = 2 * K_cache[i, j] - K_cache[i, i] - K_cache[j, j]
                if eta >= 0.0: continue

                alpha_j = alpha_j - (y_j * (E_i - E_j) / eta)
                if alpha_j > H: alpha_j = H
                if alpha_j < L: alpha_j = L
                if abs(alpha_jj - alpha_j) < tol: continue

                alpha_i = alpha_i + (y_i * y_j * (alpha_jj - alpha_j))

                b_i = b - E_i - y_i * (alpha_i - alpha_ii) * K_cache[i, i] - y_j * (alpha_j - alpha_jj) * K_cache[i, j]
                b_j = b - E_j - y_i * (alpha_i - alpha_ii) * K_cache[i, j] - y_j * (alpha_j - alpha_jj) * K_cache[j, j]

                if alpha_i > 0.0 and alpha_i < C:
                    b = b_i
                elif alpha_j > 0.0 and alpha_j < C:
                    b = b_j
                else:
                    b = (b_i + b_j) / 2.0

                alphas[i] = alpha_i
                alphas[j] = alpha_j

                num_changed_alphas = num_changed_alphas + 1

        if num_changed_alphas == 0:
            passes += 1
        else:
            passes = 0

        model = dict()
        alpha_index = [ index for index, alpha in enumerate(alphas) if alpha > 0.0 ]
        model['X'] = X[alpha_index]
        model['Y'] = Y[alpha_index]
        model['kernel'] = K
        model['alphas'] = alphas[alpha_index]
        model['b'] = b
        model['w'] = X.T * np.multiply(alphas, Y) 
        return model

In [133]:
def predict(model, X_test):
        m, n = X_test.shape
        fx = np.matrix(np.zeros([m, 1]))
#         if model['kernel'] == 'linear':
#             w = model['w']
#             b = model['b']
#             fx = X_test * w + b
#         else:
        alphas = model['alphas']
        X = model['X']
        Y = model['Y']
        w = model['w']
        K = model['kernel']
        b = model['b']
        fx = X_test * w + b
#         fx = np.multiply(np.tile(Y, [m]), rbf_kernel(X, X_test)).T * alphas + b
#         fx = np.multiply(np.tile(Y, [m]), kernel_polynomial(X, X_test)).T * alphas + b
        return fx
    
def test(model, X, Y):
        fx = predict(model,X)
        Y_pred = np.matrix(np.zeros(Y.shape))
        Y_pred[np.where(fx >= 0)] = 1
        Y_pred[np.where(fx < 0)] = -1

        P = np.matrix(np.zeros(Y.shape))
        P[np.where(Y_pred == Y)] = 1 

        return 1.0 * P.sum() / len(Y)

In [134]:
model = fit()

In [135]:
acc_train = test(model, X, Y)
acc_test = test(model, X_test, Y_test)
print('Train accuracy:', acc_train)
print('Test accuracy:', acc_test)

Train accuracy: 0.9560047562425684
Test accuracy: 0.9515279241306639


In [20]:
def plot_hyperplane(X, Y, model, plot_id, d = 500):
    I0 = np.where(Y==-1)[0]
    I1 = np.where(Y==1)[0]

    plt.subplot(plot_id)

    plt.plot(X[I1, 0], X[I1, 1], 'og')
    plt.plot(X[I0, 0], X[I0, 1], 'xb')

    min_val = np.min(X, 0)
    max_val = np.max(X, 0)

    x0_plot = np.linspace(min_val[0], max_val[0], d)
    x1_plot = np.linspace(min_val[1], max_val[1], d)

    [x0, x1] = plt.meshgrid(x0_plot, x1_plot);

    Y_all = np.matrix(np.zeros([d, d]))

    for i in range(d):
        X_all = np.matrix(np.zeros([d, 2]))
        X_all[:, 0] = np.matrix(x0[:, i]).T
        X_all[:, 1] = np.matrix(x1[:, i]).T
        Y_all[:, i] = predict(model,X_all)

    plt.contour(np.array(x0), np.array(x1), np.array(Y_all), levels = [0.0], colors = 'red')

In [21]:
#  plot_hyperplane(X_test, Y_test,model, 231)